# Initializations
- Dataset columns should include "ProteinID" and "ModifiedLocationNum" 

In [ ]:
from protds.protds_alphafold import *

alphafold_download_folder("/home/skduong/Desktop/prots/downloads/") #specify where Alphafold downloads are

filename = 'Peptide_IndexByID_Version_test.csv'
path = r'data' #can replace "data" with any path 
data = pd.read_csv(os.path.join(path ,filename))
df = data[pd.notna(data.ModifiedLocationNum)].copy() #keep only rows with ModifiedLocationNumber

# Distances
- unit measure is 1 Angstrom
- for each Feature site, distance is defined by the euclidean distance between centerOfMass(ModifiedLocation) and centerOfMass(siteLocation)

In [ ]:
#Calculate Distances (takes a while)
df["Feature"] = [[] for i in df.index]
df["Distance_to_ModifiedLocationNum"] = [[] for i in df.index]
dist_list = []
feats_list = []

for i in df.index:
    entry = df.loc[i]
    feats, dists = get_distance_to_features(entry.ProteinID, entry.ModifiedLocationNum)
    dist_list.append(dists)
    feats_list.append(feats)
    
df["Feature"] = feats_list
df["Distance_to_ModifiedLocationNum"] = dist_list

df_long = df.explode(column=["Feature", "Distance_to_ModifiedLocationNum"])

# Output
- summary and long tables saved in the output folder

In [ ]:
#Save Results
output_path = r'output' #location of output directory
df.to_csv(os.path.join(output_path, ''.join(filename.split('.')[:-1])+"-summary.csv"), index=False)
df_long.to_csv(os.path.join(output_path, ''.join(filename.split('.')[:-1])+"-long.csv"), index=False)